In [3]:
import pandas as pd

train_set = pd.read_csv('data/병합데이터셋-v5-bt.csv', index_col=0)

train_set = train_set.dropna()
train_set = train_set.reset_index(drop=True)

# 감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5 }
train_set['emotion'] = train_set.emotion.map(emotions)

# 데이터 프레임 셔플
train_set = train_set.sample(frac=1).reset_index(drop=True)
train_set.shape

(147525, 2)

In [4]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import gluonnlp as nlp

#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = torch.device("cuda")

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 7  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 2e-5

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]         # self.labels = [np.int32(i[label_idx]) for i in dataset] 이꺼 변경



    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [5]:
train_accuarcy, test_accuarcy = [], []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train_accuarcy.append(train_acc / (batch_id+1))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    test_accuarcy.append(test_acc / (batch_id+1))
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
print(train_accuarcy, test_accuarcy)

  0%|          | 0/3689 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7849496603012085 train acc 0.28125
epoch 1 batch id 201 loss 1.7854769229888916 train acc 0.22046019900497513
epoch 1 batch id 401 loss 1.5321274995803833 train acc 0.26317019950124687
epoch 1 batch id 601 loss 1.5046125650405884 train acc 0.30870424292845255
epoch 1 batch id 801 loss 1.187895655632019 train acc 0.3533863920099875
epoch 1 batch id 1001 loss 1.410932183265686 train acc 0.38920454545454547
epoch 1 batch id 1201 loss 1.1485865116119385 train acc 0.41980641132389673
epoch 1 batch id 1401 loss 1.1155807971954346 train acc 0.4452177016416845
epoch 1 batch id 1601 loss 1.1803524494171143 train acc 0.46720799500312304
epoch 1 batch id 1801 loss 1.0862274169921875 train acc 0.48481746252082175
epoch 1 batch id 2001 loss 0.9968980550765991 train acc 0.5001717891054472
epoch 1 batch id 2201 loss 1.055002212524414 train acc 0.5129060654248069
epoch 1 batch id 2401 loss 0.8070299029350281 train acc 0.524039462723865
epoch 1 batch id 2601 loss 0.92052936553

  0%|          | 0/923 [00:00<?, ?it/s]

epoch 1 test acc 0.6718242145178764


  0%|          | 0/3689 [00:00<?, ?it/s]

...

epoch 6 test acc 0.6869582881906826


  0%|          | 0/3689 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.16904717683792114 train acc 0.9375
epoch 7 batch id 201 loss 0.5248503684997559 train acc 0.8535447761194029
epoch 7 batch id 401 loss 0.5631686449050903 train acc 0.8583229426433915
epoch 7 batch id 601 loss 0.23955106735229492 train acc 0.8577891014975042
epoch 7 batch id 801 loss 0.21588607132434845 train acc 0.8588483146067416
epoch 7 batch id 1001 loss 0.3903769850730896 train acc 0.8575487012987013
epoch 7 batch id 1201 loss 0.39481863379478455 train acc 0.8565258118234804
epoch 7 batch id 1401 loss 0.42969661951065063 train acc 0.8575347965738758
epoch 7 batch id 1601 loss 0.6024450659751892 train acc 0.8580379450343535
epoch 7 batch id 1801 loss 0.5603017807006836 train acc 0.8592448639644642
epoch 7 batch id 2001 loss 0.33036181330680847 train acc 0.8606009495252374
epoch 7 batch id 2201 loss 0.4992963671684265 train acc 0.8608445024988641
epoch 7 batch id 2401 loss 0.17254726588726044 train acc 0.8616201582673886
epoch 7 batch id 2601 loss 0.61826884

  0%|          | 0/923 [00:00<?, ?it/s]

epoch 7 test acc 0.6858410075839654
3190.03125 633.03125


In [1]:
import pandas as pd

train_set = pd.read_csv('data/병합데이터셋-v5-bt.csv', index_col=0)

train_set = train_set.dropna()
train_set = train_set.reset_index(drop=True)

# 감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5 }
train_set['emotion'] = train_set.emotion.map(emotions)

# 데이터 프레임 셔플
train_set = train_set.sample(frac=1).reset_index(drop=True)
train_set.shape

(147525, 2)

In [2]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import gluonnlp as nlp

#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = torch.device("cuda")

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 7  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]         # self.labels = [np.int32(i[label_idx]) for i in dataset] 이꺼 변경



    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [3]:
train_accuarcy, test_accuarcy = [], []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train_accuarcy.append(train_acc / (batch_id+1))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    test_accuarcy.append(test_acc / (batch_id+1))
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
print(train_accuarcy, test_accuarcy, sep='\n')

  0%|          | 0/3689 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8258147239685059 train acc 0.15625
epoch 1 batch id 201 loss 1.6981139183044434 train acc 0.1957400497512438
epoch 1 batch id 401 loss 1.7189544439315796 train acc 0.24251870324189526
epoch 1 batch id 601 loss 1.6414874792099 train acc 0.2849417637271215
epoch 1 batch id 801 loss 1.4733942747116089 train acc 0.325374531835206
epoch 1 batch id 1001 loss 1.1684176921844482 train acc 0.35967157842157843
epoch 1 batch id 1201 loss 1.3892159461975098 train acc 0.38782785179017487
epoch 1 batch id 1401 loss 1.0611392259597778 train acc 0.41303087080656675
epoch 1 batch id 1601 loss 1.136086106300354 train acc 0.4338694565896315
epoch 1 batch id 1801 loss 1.4218536615371704 train acc 0.45164144919489174
epoch 1 batch id 2001 loss 1.2108484506607056 train acc 0.46842203898050977
epoch 1 batch id 2201 loss 0.9725021719932556 train acc 0.48225238527941844
epoch 1 batch id 2401 loss 1.0816774368286133 train acc 0.4947808204914619
epoch 1 batch id 2601 loss 0.907434225082

  0%|          | 0/923 [00:00<?, ?it/s]

epoch 1 test acc 0.6603805525460456


  0%|          | 0/3689 [00:00<?, ?it/s]

...

epoch 6 test acc 0.6914612676056338


  0%|          | 0/3689 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.6601365804672241 train acc 0.75
epoch 7 batch id 201 loss 0.6161670684814453 train acc 0.8022388059701493
epoch 7 batch id 401 loss 0.5875775814056396 train acc 0.7997194513715711
epoch 7 batch id 601 loss 0.6128020286560059 train acc 0.7995528286189684
epoch 7 batch id 801 loss 0.3814089298248291 train acc 0.7992743445692884
epoch 7 batch id 1001 loss 0.3913452625274658 train acc 0.7988573926073926
epoch 7 batch id 1201 loss 0.7159830927848816 train acc 0.7969660699417153
epoch 7 batch id 1401 loss 0.459958553314209 train acc 0.7977560670949322
epoch 7 batch id 1601 loss 0.4526872932910919 train acc 0.7987781074328545
epoch 7 batch id 1801 loss 0.8124911189079285 train acc 0.7989137978900611
epoch 7 batch id 2001 loss 0.5914962291717529 train acc 0.7997876061969016
epoch 7 batch id 2201 loss 0.37870484590530396 train acc 0.8002896410722399
epoch 7 batch id 2401 loss 0.6466255784034729 train acc 0.8005648688046647
epoch 7 batch id 2601 loss 0.47367438673973083

  0%|          | 0/923 [00:00<?, ?it/s]

epoch 7 test acc 0.6932218309859155
[0.5489123068582271, 0.6846960558416915, 0.722011385199241, 0.7529903090268365, 0.7781749796692871, 0.7951511249661155, 0.8026819598807264] [0.6603805525460456, 0.6875338569880823, 0.6896329902491874, 0.6885495666305526, 0.6914274106175514, 0.6914612676056338, 0.6932218309859155]


In [3]:
import pandas as pd

train_set = pd.read_csv('data/병합데이터셋-v5.csv', index_col=0)

train_set = train_set.dropna()
train_set = train_set.reset_index(drop=True)

# 감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5 }
train_set['emotion'] = train_set.emotion.map(emotions)

# 데이터 프레임 셔플
train_set = train_set.sample(frac=1).reset_index(drop=True)
train_set.shape

# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import gluonnlp as nlp

#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = torch.device("cuda")

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 7  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 2e-5

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]         # self.labels = [np.int32(i[label_idx]) for i in dataset] 이꺼 변경



    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.1, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

train_accuarcy, test_accuarcy = [], []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train_accuarcy.append(train_acc / (batch_id+1))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    test_accuarcy.append(test_acc / (batch_id+1))
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
train_accuarcy, test_accuarcy

  0%|          | 0/4160 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9032812118530273 train acc 0.0625
epoch 1 batch id 201 loss 1.751467227935791 train acc 0.17273009950248755
epoch 1 batch id 401 loss 1.6798760890960693 train acc 0.23986907730673318
epoch 1 batch id 601 loss 1.4346672296524048 train acc 0.2873336106489185
epoch 1 batch id 801 loss 1.0671095848083496 train acc 0.33941947565543074
epoch 1 batch id 1001 loss 1.075100064277649 train acc 0.37609265734265734
epoch 1 batch id 1201 loss 1.2916501760482788 train acc 0.40825353871773523
epoch 1 batch id 1401 loss 1.2795116901397705 train acc 0.4351133119200571
epoch 1 batch id 1601 loss 0.9827401041984558 train acc 0.4555356027482823
epoch 1 batch id 1801 loss 0.8462985157966614 train acc 0.47494447529150474
epoch 1 batch id 2001 loss 0.9094249606132507 train acc 0.49094202898550726
epoch 1 batch id 2201 loss 0.8161762952804565 train acc 0.5043588141753749
epoch 1 batch id 2401 loss 0.8118577599525452 train acc 0.517219387755102
epoch 1 batch id 2601 loss 0.69574749469

  0%|          | 0/463 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.22 GiB already allocated; 7.56 MiB free; 4.50 GiB reserved in total by PyTorch)

In [ ]:
torch.save(model, 'model/kobert-v10.pt')

In [ ]:
# 모델 사이즈 확인(파라미터는 v1과 동일)
import os

model_path3 = 'model/kobert-v3.pt'
size2 = os.path.getsize(model_path3) / (1024*1024) # mb 단위
print(f"Model size: {size2:.2f} MB")